<a href="https://colab.research.google.com/github/Pathway2008/Loan_rating/blob/main/blender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/loan_rating/train.csv')
test = pd.read_csv('/content/drive/MyDrive/loan_rating/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/loan_rating/sample_submission.csv')

In [3]:
train.drop(['ID'],axis=1,inplace=True)
test.drop(['ID'],axis=1,inplace=True)

In [ ]:
train = train[train['주택소유상태'] != 'ANY']
train['근로기간'] = train['근로기간'].replace('< 1 year', '<1 year')
train['근로기간'] = train['근로기간'].replace('1 year', '1 years')
train['근로기간'] = train['근로기간'].replace('3', '3 years')
train['근로기간'] = train['근로기간'].replace('10+years', '10+ years')

In [7]:
test['근로기간'] = test['근로기간'].replace('< 1 year', '<1 year')
test['근로기간'] = test['근로기간'].replace('1 year', '1 years')
test['근로기간'] = test['근로기간'].replace('3', '3 years')
test['근로기간'] = test['근로기간'].replace('10+years', '10+ years')

In [8]:
train['원금/대출'] = train['총상환원금']/train['대출금액']
train['이자/대출'] = train['총상환이자']/train['대출금액']
train['소득/대출'] = train['연간소득'] / train['대출금액']
train['연체계좌/총계좌'] = train['연체계좌수'] / train['총계좌수']

In [9]:
train['대출월금액'] = train.apply(lambda row: row['대출금액'] / 36 if row['대출기간'] == 0 else row['대출금액'] / 60, axis=1)
train['이자/소득'] = train.apply(lambda row: row['총상환이자'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
train['원금/소득'] = train.apply(lambda row: row['총상환원금'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
train['이자/원금'] = train.apply(lambda row: row['총상환이자'] / row['총상환원금'] if row['총상환원금'] != 0 else 0, axis=1)

In [74]:
test['원금/대출'] = test['총상환원금'] / test['대출금액']
test['이자/대출'] = test['총상환이자'] / test['대출금액']
test['소득/대출'] = test['연간소득'] / test['대출금액']
test['연체계좌/총계좌'] = test['연체계좌수'] / test['총계좌수']
test['대출월금액'] = test.apply(lambda row: row['대출금액'] / 36 if row['대출기간'] == 0 else row['대출금액'] / 60, axis=1)
test['이자/소득'] = test.apply(lambda row: row['총상환이자'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
test['원금/소득'] = test.apply(lambda row: row['총상환원금'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
test['이자/원금'] = test.apply(lambda row: row['총상환이자'] / row['총상환원금'] if row['총상환원금'] != 0 else 0, axis=1)

###add

In [11]:
train['총상환이자**3'] = train['총상환이자']**3
train['총상환원금**3'] = train['총상환원금']**3

test['총상환이자**3'] = test['총상환이자']**3
test['총상환원금**3'] = test['총상환원금']**3

In [12]:
from sklearn.preprocessing import LabelEncoder
ordinal_features = ['대출기간', '근로기간', '주택소유상태','대출목적']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [13]:
train['대출등급'] = le.fit_transform(train['대출등급'])

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96293 entries, 0 to 96293
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          96293 non-null  int64  
 1   대출기간          96293 non-null  int64  
 2   근로기간          96293 non-null  int64  
 3   주택소유상태        96293 non-null  int64  
 4   연간소득          96293 non-null  int64  
 5   부채_대비_소득_비율   96293 non-null  float64
 6   총계좌수          96293 non-null  int64  
 7   대출목적          96293 non-null  int64  
 8   최근_2년간_연체_횟수  96293 non-null  int64  
 9   총상환원금         96293 non-null  int64  
 10  총상환이자         96293 non-null  float64
 11  총연체금액         96293 non-null  float64
 12  연체계좌수         96293 non-null  float64
 13  대출등급          96293 non-null  int64  
 14  원금/대출         96293 non-null  float64
 15  이자/대출         96293 non-null  float64
 16  소득/대출         96293 non-null  float64
 17  연체계좌/총계좌      96293 non-null  float64
 18  대출월금액         96293 non-nu

In [15]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00


In [16]:
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [17]:
from sklearn.model_selection import KFold
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [60]:
from sklearn.model_selection import train_test_split
X = train.drop(['대출등급','총상환원금','총상환이자'], axis=1)
y = train['대출등급']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)

In [61]:
# XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
xgb_predictions = xgb_model.predict(X_test)
xgb_macro = f1_score(y_test, xgb_predictions, average='macro')
# print(f'XGBoost test F1 Macro: {xgb_macro}\n')


# CatBoost
cat_model = CatBoostClassifier(random_state=42, verbose=False)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)
cat_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
cat_macro = f1_score(y_test, cat_predictions, average='macro')
# print(f'Cat test F1 Macro: {cat_macro}\n')

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
rf_macro = f1_score(y_test, rf_predictions, average='macro')
# print(f'RF test F1 Macro: {rf_macro}\n')

# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')
lgb_macro = f1_score(y_test, lgb_predictions, average='macro')
# print(f'RF test F1 Macro: {lgb_macro}\n')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3284
[LightGBM] [Info] Number of data points in the train set: 67405, number of used features: 21
[LightGBM] [Info] Start training from score -1.747717
[LightGBM] [Info] Start training from score -1.206424
[LightGBM] [Info] Start training from score -1.248802
[LightGBM] [Info] Start training from score -1.975557
[LightGBM] [Info] Start training from score -2.572111
[LightGBM] [Info] Start training from score -3.897369
[LightGBM] [Info] Start training from score -5.434895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006095 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3271
[LightGBM

In [20]:
#총상환원금/이자**3
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9158510962734695
CatBoost Cross-Validation F1 Macro: 0.899137891181676
Random Forest Cross-Validation F1 Macro: 0.9055145861483181
LightGBM Cross-Validation F1 Macro: 0.8879400699925888


xgb test F1 Macro: 0.9126216602273062
cat test F1 Macro: 0.893209437350842
rf test F1 Macro: 0.9086610022835965
lgb test F1 Macro: 0.8583661222202089


In [62]:
#except 총상환원금/이자
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9160147491711779
CatBoost Cross-Validation F1 Macro: 0.9008756511076592
Random Forest Cross-Validation F1 Macro: 0.9099087706541467
LightGBM Cross-Validation F1 Macro: 0.8893886206105155


xgb test F1 Macro: 0.9121720641610676
cat test F1 Macro: 0.8920197440554183
rf test F1 Macro: 0.9109011643763604
lgb test F1 Macro: 0.8805477229272579


In [63]:
cat_predictions_flat = np.ravel(cat_predictions)
data = {
    'xgb': xgb_predictions,
    'cat': cat_predictions_flat,
    'lgb': lgb_predictions,
    'rf': rf_predictions,
    'sol': y_test

}
pred_df = pd.DataFrame(data)

In [64]:
#최빈값 대체
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

In [65]:
stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9105294743631511

In [71]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64197 entries, 0 to 64196
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          64197 non-null  int64  
 1   대출기간          64197 non-null  int64  
 2   근로기간          64197 non-null  int64  
 3   주택소유상태        64197 non-null  int64  
 4   연간소득          64197 non-null  int64  
 5   부채_대비_소득_비율   64197 non-null  float64
 6   총계좌수          64197 non-null  int64  
 7   대출목적          64197 non-null  int64  
 8   최근_2년간_연체_횟수  64197 non-null  int64  
 9   총상환원금         64197 non-null  int64  
 10  총상환이자         64197 non-null  float64
 11  총연체금액         64197 non-null  float64
 12  연체계좌수         64197 non-null  float64
 13  원금/대출         64197 non-null  float64
 14  이자/대출         64197 non-null  float64
 15  대출월금액         64197 non-null  float64
 16  소득/대출         64197 non-null  float64
 17  이자/소득         64197 non-null  float64
 18  원금/소득         64197 non-nu

In [77]:
test = test[['대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총연체금액', '연체계좌수', '원금/대출', '이자/대출', '소득/대출', '연체계좌/총계좌', '대출월금액', '이자/소득', '원금/소득', '이자/원금', '총상환이자**3', '총상환원금**3']]

In [79]:
xgb_pred = xgb_model.predict(test)
cat_pred = cat_model.predict(test)
rf_pred = rf_model.predict(test)
lgb_pred = lgb_model.predict(test)

In [80]:
cat_predictions_flat = np.ravel(cat_pred)
data = {
    'xgb': xgb_pred,
    'cat': cat_predictions_flat,
    'lgb': lgb_pred,
    'rf': rf_pred,

}
pred_blender = pd.DataFrame(data)

In [81]:
#최빈값 대체
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']

pred_blender['pred'] = pred_blender[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_blender['pred'] = pred_blender['pred'].astype(int)

In [84]:
sub['대출등급'] = pred_blender['pred']
sub['대출등급'] = sub['대출등급'].map({0: 'A', 1: 'B', 2: 'C',3: 'D',4: 'E',5: 'F',6: 'G'})
sub.head()

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C


In [85]:
sub.to_csv('blender.csv',index=False)

#standard scaling

In [24]:
from sklearn.preprocessing import StandardScaler

# Extract the columns to be scaled
columns_to_scale = [col for col in X_train.columns if col not in ['대출기간', '근로기간', '주택소유상태', '대출목적']]

# Create a MinMaxScaler
scaler = StandardScaler()

# Fit and transform the selected columns in X_train
X_train_scaled = X_train.copy()
X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])

# Transform the selected columns in X_test
X_test_scaled = X_test.copy()
X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

In [53]:
# XGBoost
xgb_model = xgb.XGBClassifier(
    random_state=42,
    learning_rate=0.1,  # Adjust as needed
    max_depth=7,         # Adjust as needed
    n_estimators=100     # Adjust as needed
)
xgb_model.fit(X_train_scaled, y_train)
xgb_scores = cross_val_score(xgb_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
xgb_predictions = xgb_model.predict(X_test_scaled)
xgb_macro = f1_score(y_test, xgb_predictions, average='macro')
# print(f'XGBoost test F1 Macro: {xgb_macro}\n')


# CatBoost
cat_model = CatBoostClassifier(
    depth= 8,
    iterations = 300,
    learning_rate= 0.2,
    random_state=42, verbose=False)
cat_model.fit(X_train_scaled, y_train)
cat_predictions = cat_model.predict(X_test_scaled)
cat_scores = cross_val_score(cat_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
cat_macro = f1_score(y_test, cat_predictions, average='macro')
# print(f'Cat test F1 Macro: {cat_macro}\n')

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)
rf_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
rf_macro = f1_score(y_test, rf_predictions, average='macro')
# print(f'RF test F1 Macro: {rf_macro}\n')

# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train_scaled, y_train)
lgb_predictions = lgb_model.predict(X_test_scaled)
lgb_scores = cross_val_score(lgb_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')
lgb_macro = f1_score(y_test, lgb_predictions, average='macro')
# print(f'RF test F1 Macro: {lgb_macro}\n')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009023 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3288
[LightGBM] [Info] Number of data points in the train set: 67405, number of used features: 21
[LightGBM] [Info] Start training from score -1.747717
[LightGBM] [Info] Start training from score -1.206424
[LightGBM] [Info] Start training from score -1.248802
[LightGBM] [Info] Start training from score -1.975557
[LightGBM] [Info] Start training from score -2.572111
[LightGBM] [Info] Start training from score -3.897369
[LightGBM] [Info] Start training from score -5.434895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005843 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3275
[LightGBM

In [26]:
#standard scaling
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9156006885694543
CatBoost Cross-Validation F1 Macro: 0.9019676380300858
Random Forest Cross-Validation F1 Macro: 0.9120258824601837
LightGBM Cross-Validation F1 Macro: 0.9038266615917709


xgb test F1 Macro: 0.9121720641610676
cat test F1 Macro: 0.8944004429576997
rf test F1 Macro: 0.9088630411104687
lgb test F1 Macro: 0.8719019896219197


In [57]:
cat_predictions_flat = np.ravel(cat_predictions)
data = {
    'xgb': xgb_predictions,
    'cat': cat_predictions_flat,
    'lgb': lgb_predictions,
    'rf': rf_predictions,
    'sol': y_test

}
pred_df = pd.DataFrame(data)

In [37]:
rows_with_different_values = pred_df[pred_df.apply(lambda row: row.nunique() > 1, axis=1)]
rows_with_different_values

,xgb,cat,lgb,rf,sol,pred
42625,0,0,1,1,0,0
32728,0,0,1,0,0,0
52191,1,2,1,1,1,1
17484,2,1,1,1,1,1
58966,1,1,0,1,1,1
...,...,...,...,...,...,...
22887,1,1,1,4,2,1
62552,2,2,2,2,1,2
55856,0,1,0,0,0,0
21738,0,1,1,1,1,1


In [47]:
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']
mode_count_per_row = pred_df[prediction_columns].apply(lambda row: row.mode().count(), axis=1)

mode_2 = pred_df[mode_count_per_row >= 2]
mode_2

,xgb,cat,lgb,rf,sol,pred
42625,0,0,1,1,0,0
46409,0,1,0,1,0,0
968,1,1,0,0,1,0
17913,3,2,3,2,2,2
31062,0,0,1,1,0,0
...,...,...,...,...,...,...
30850,1,1,2,2,1,1
46986,4,3,4,3,3,3
6578,1,1,2,2,1,1
77442,0,2,2,0,2,0


In [48]:
mode_xgb = f1_score(mode_2['sol'], mode_2['xgb'], average='macro')
mode_cat = f1_score(mode_2['sol'], mode_2['cat'], average='macro')
mode_lgb = f1_score(mode_2['sol'], mode_2['lgb'], average='macro')
mode_rf = f1_score(mode_2['sol'], mode_2['rf'], average='macro')
mode_xgb, mode_cat, mode_lgb, mode_rf

(0.4720812920410721,
 0.2824755081665651,
 0.37413643124066637,
 0.4292266850791379)

In [49]:
#최빈값 대체
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

In [50]:
pred_df.head()

,xgb,cat,lgb,rf,sol,pred
72084,2,2,2,2,2,2
42625,0,0,1,1,0,0
1288,1,1,1,1,1,1
19240,0,0,0,0,0,0
10764,2,2,2,2,2,2


In [52]:
stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9139110999584489

In [54]:
#standard scaling + xgb,cat hyper
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9174397986766658
CatBoost Cross-Validation F1 Macro: 0.8981622064275185
Random Forest Cross-Validation F1 Macro: 0.9120258824601837
LightGBM Cross-Validation F1 Macro: 0.9038266615917709


xgb test F1 Macro: 0.9117522663658518
cat test F1 Macro: 0.8934628492843932
rf test F1 Macro: 0.9088630411104687
lgb test F1 Macro: 0.8719019896219197


In [58]:
#최빈값 대체
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

In [59]:
stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9128732753277623